Pré-requis
Installe la librairie pour gérer les accents si ce n'est pas fait :

In [1]:
pip install unidecode

Note: you may need to restart the kernel to use updated packages.


Cellule 1 : Imports et Configuration
On ajoute numpy et unidecode pour le nettoyage.

In [2]:
import json
import pandas as pd
import numpy as np
import os
from unidecode import unidecode # Pour enlever les accents

# Fonction de nettoyage de texte (Règle 2, 3, 4, 7)
def clean_text(text):
    if pd.isna(text) or text == "":
        return "unknown"
    # Minuscule + Sans accent + Strip
    return unidecode(str(text)).lower().strip()

print("✅ Bibliothèques chargées et fonctions prêtes.")

✅ Bibliothèques chargées et fonctions prêtes.


Cellule 3 : Extraction (Extract)
Ici, on récupère les données brutes. Note importante : Je renomme immédiatement les colonnes pour qu'elles correspondent à tes règles (ex: home_result du JSON devient home_goals).
code


In [3]:
# 1. Chargement
with open('data/data_2018.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 2. Mappings (Lookups)
teams_map = {t['id']: t['name'] for t in data['teams']}
stadiums_map = {s['id']: s['city'] for s in data['stadiums']}

# 3. Extraction
all_matches = []

# Fonction interne pour traiter une liste de matchs
def process_matches(match_list, round_name):
    for m in match_list:
        all_matches.append({
            'date': m['date'],
            'round': round_name,
            'city': stadiums_map.get(m['stadium']),
            'home_team_name': teams_map.get(m['home_team']),
            'away_team_name': teams_map.get(m['away_team']),
            'home_goals': m['home_result'], # Attention : Dans le JSON c'est le score
            'away_goals': m['away_result'], # Attention : Dans le JSON c'est le score
            'edition': 2018
        })

# Boucle Groupes
for k, v in data['groups'].items():
    process_matches(v['matches'], v['name'])

# Boucle Finales
for k, v in data['knockout'].items():
    process_matches(v['matches'], v['name'])

# Création du DataFrame brut
df = pd.DataFrame(all_matches)

print(f"📊 Extraction terminée : {len(df)} matchs récupérés.")
display(df.head(3))

📊 Extraction terminée : 64 matchs récupérés.


,date,round,city,home_team_name,away_team_name,home_goals,away_goals,edition
0,2018-06-14T18:00:00+03:00,Group A,Moscow,Russia,Saudi Arabia,5,0,2018
1,2018-06-15T17:00:00+05:00,Group A,Yekaterinburg,Egypt,Uruguay,0,1,2018
2,2018-06-19T21:00:00+03:00,Group A,Saint Petersburg,Russia,Egypt,3,1,2018


Cellule 4 : Transformation & Nettoyage (Transform)
C'est ici qu'on applique tes règles ETL_RULES.md et qu'on corrige ton erreur de date.

In [5]:
print("🧹 Démarrage du nettoyage selon ETL_RULES.md...")

# --- RÈGLE 1 : DATE ---
# Correction de ton erreur : On convertit en datetime, puis on formate en string ISO
# Format demandé : YYYY-MM-DD hh:mm:ss
df['date'] = pd.to_datetime(df['date'], utc=True).dt.strftime('%Y-%m-%d %H:%M:%S')

# --- RÈGLE 3, 4, 7 : TEXTE (Villes, Equipes) ---
# Minuscule, sans accents
text_cols = ['city', 'home_team_name', 'away_team_name']
for col in text_cols:
    df[col] = df[col].apply(clean_text)

# --- RÈGLE 2 : ROUND (Standardisation) ---
df['round'] = df['round'].apply(clean_text)

def standardize_round(val):
    if 'group' in val: return 'group' # Remplace "group a" par "group"
    if 'round of 16' in val: return 'round of 16'
    if 'quarter' in val: return 'quarter-final'
    if 'semi' in val: return 'semi-final'
    if 'third' in val: return 'play-off for third place'
    if 'final' in val: return 'final'
    return val

df['round'] = df['round'].apply(standardize_round)

# --- RÈGLE 5, 6 : GOALS (Entiers) ---
df['home_goals'] = df['home_goals'].fillna(0).astype(int)
df['away_goals'] = df['away_goals'].fillna(0).astype(int)

# --- RÈGLE 8, 9 : CALCUL DU RÉSULTAT (Winner/Loser) ---
# On recrée la colonne home_result et away_result mais avec "winner"/"loser"
conditions = [
    (df['home_goals'] > df['away_goals']),
    (df['home_goals'] < df['away_goals']),
    (df['home_goals'] == df['away_goals'])
]
choices_home = ['winner', 'loser', 'draw']
choices_away = ['loser', 'winner', 'draw']

df['home_result'] = np.select(conditions, choices_home, default='draw')
df['away_result'] = np.select(conditions, choices_away, default='draw')

# Réorganisation des colonnes pour faire propre
cols_order = ['date', 'edition', 'round', 'city', 
              'home_team_name', 'away_team_name', 
              'home_goals', 'away_goals', 
              'home_result', 'away_result']
df = df[cols_order]

print("✨ Nettoyage terminé !")

🧹 Démarrage du nettoyage selon ETL_RULES.md...
✨ Nettoyage terminé !


Cellule 5 : Affichage Final

In [11]:
# --- ÉTAPE FINALE : RENOMMAGE ET MISE EN FORME ---

# 1. Dictionnaire de renommage (Ancien nom -> Nouveau nom)
rename_mapping = {
    'date': 'Date',
    'round': 'Stage',
    'city': 'City',
    'home_team_name': 'Home Team Name',
    'away_team_name': 'Away Team Name',
    'home_goals': 'Home Team Goals',
    'away_goals': 'Away Team Goals',
    'home_result': 'Home Result',
    'away_result': 'Away Result'
}

# 2. Application du renommage
df = df.rename(columns=rename_mapping)

# 3. Sélection et Ordre des colonnes (On supprime 'edition' ici en ne le sélectionnant pas)
final_cols = [
    'Date', 
    'Stage', 
    'City', 
    'Home Team Name', 
    'Away Team Name', 
    'Home Team Goals', 
    'Away Team Goals', 
    'Home Result', 
    'Away Result'
]

df = df[final_cols]
print(f"📋 Tableau Final ({len(df)} matchs) :")
print("✨ Tableau finalisé avec les bons noms de colonnes :")
display(df)

📋 Tableau Final (64 matchs) :
✨ Tableau finalisé avec les bons noms de colonnes :


,Date,Stage,City,Home Team Name,Away Team Name,Home Team Goals,Away Team Goals,Home Result,Away Result
0,2018-06-14 15:00:00,group,moscow,russia,saudi arabia,5,0,winner,loser
1,2018-06-15 12:00:00,group,yekaterinburg,egypt,uruguay,0,1,loser,winner
2,2018-06-19 18:00:00,group,saint petersburg,russia,egypt,3,1,winner,loser
3,2018-06-20 15:00:00,group,rostov-on-don,uruguay,saudi arabia,1,0,winner,loser
4,2018-06-25 14:00:00,group,samara,uruguay,russia,3,0,winner,loser
5,2018-06-25 14:00:00,group,volgograd,saudi arabia,egypt,2,1,winner,loser
6,2018-06-15 18:00:00,group,sochi,portugal,spain,3,3,draw,draw
7,2018-06-15 15:00:00,group,saint petersburg,morocco,iran,0,1,loser,winner
8,2018-06-20 12:00:00,group,moscow,portugal,morocco,1,0,winner,loser
9,2018-06-20 18:00:00,group,kazan,iran,spain,0,1,loser,winner
